In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [2]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Mango/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Mango/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Mango/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 41 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 13 images belonging to 1 classes.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
2/2 [==============================] - 11s 7s/step - loss: 2.3247 - accuracy: 0.5610 - val_loss: 0.9905 - val_accuracy: 0.5000
Epoch 2/5
2/2 [==============================] - 4s 2s/step - loss: 1.0402 - accuracy: 0.5122 - val_loss: 0.7114 - val_accuracy: 0.5000
Epoch 3/5
2/2 [==============================] - 5s 4s/step - loss: 0.5510 - accuracy: 0.8537 - val_loss: 0.3976 - val_accuracy: 0.9000
Epoch 4/5
2/2 [==============================] - 4s 3s/step - loss: 0.3752 - accuracy: 0.8049 - val_loss: 0.4159 - val_accuracy: 0.6000
Epoch 5/5
2/2 [==============================] - 6s 5s/step - loss: 0.1729 - accuracy: 0.9756 - val_loss: 0.0149 - val_accuracy: 1.0000


In [7]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

#load the model
model = load_model('/content/Model.h5')
#classes
class_names = ['Raw Mango','Ripen Mango']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [9]:
image = Image.open("/content/drive/MyDrive/Mango/test/Medium Mangoes/10.jpeg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

In [11]:
# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
# Print prediction and confidence score
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)

1/1 [==============================] - 0s 152ms/step
Result:  Raw Mango

Accuracy:  nan
